# Leetcode Premium Problems

In [2]:
from typing import List
from typing import Optional
import pandas as pd
import numpy as np

## 1757. Recyclable and Low Fat Products [E]

Table: Products
```
+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| product_id  | int     |
| low_fats    | enum    |
| recyclable  | enum    |
+-------------+---------+
product_id is the primary key (column with unique values) for this table.
low_fats is an ENUM (category) of type ('Y', 'N') where 'Y' means this product is low fat and 'N' means it is not.
recyclable is an ENUM (category) of types ('Y', 'N') where 'Y' means this product is recyclable and 'N' means it is not.
```

Write a solution to find the ids of products that are both low fat and recyclable.

Return the result table in any order.

The result format is in the following example.



In [6]:
# Beats 33.89%
def find_products(products: pd.DataFrame) -> pd.DataFrame:
    return products.loc[ (products['low_fats']=='Y') & (products['recyclable']=='Y'), ['product_id'] ]

In [3]:
data = [['0', 'Y', 'N'], ['1', 'Y', 'Y'], ['2', 'N', 'Y'], ['3', 'Y', 'Y'], ['4', 'N', 'N']]
products = pd.DataFrame(data, columns=['product_id', 'low_fats', 'recyclable']).astype({'product_id':'int64', 'low_fats':'category', 'recyclable':'category'})

In [9]:
find_products(products)

,product_id


## 2877. Create a DataFrame from List [E]

Write a solution to create a DataFrame from a 2D list called `student_data`. This 2D list contains the IDs and ages of some students.

The DataFrame should have two columns, `student_id` and `age`, and be in the same order as the original 2D list.

The result format is in the following example.

In [12]:
# Beats 5.06%
def createDataframe(student_data: List[List[int]]) -> pd.DataFrame:
    return pd.DataFrame(student_data, columns=['student_id', 'age'])

In [10]:
student_data = [
  [1, 15],
  [2, 11],
  [3, 11],
  [4, 20]
]

In [13]:
createDataframe(student_data)

,student_id,age
0,1,15
1,2,11
2,3,11
3,4,20


## 1661. Average Time of Process per Machine [E]

Table: Activity
```
+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| machine_id     | int     |
| process_id     | int     |
| activity_type  | enum    |
| timestamp      | float   |
+----------------+---------+
The table shows the user activities for a factory website.
(machine_id, process_id, activity_type) is the primary key (combination of columns with unique values) of this table.
machine_id is the ID of a machine.
process_id is the ID of a process running on the machine with ID machine_id.
activity_type is an ENUM (category) of type ('start', 'end').
timestamp is a float representing the current time in seconds.
'start' means the machine starts the process at the given timestamp and 'end' means the machine ends the process at the given timestamp.
The 'start' timestamp will always be before the 'end' timestamp for every (machine_id, process_id) pair.
It is guaranteed that each (machine_id, process_id) pair has a 'start' and 'end' timestamp.
```

There is a factory website that has several machines each running the same number of processes. Write a solution to find the average time each machine takes to complete a process.

The time to complete a process is the `'end' timestamp` minus the `'start' timestamp`. The average time is calculated by the total time to complete every process on the machine divided by the number of processes that were run.

The resulting table should have the `machine_id` along with the average time as `processing_time`, which should be rounded to 3 decimal places.

Return the result table in any order.

The result format is in the following example.

In [73]:
# Beats 33.89%
def get_average_time(activity: pd.DataFrame) -> pd.DataFrame:
    if activity.shape[0] < 1:
        return pd.DataFrame(columns=['machine_id','processing_time'])
    df = activity.pivot(index=['machine_id','process_id'], columns=['activity_type'], values=['timestamp']).reset_index().droplevel(level=1, axis=1)
    df.columns = ['machine_id','process_id','end','start']
    df['ttime'] = df['end'] - df['start']
    return df.groupby('machine_id', as_index=False).agg(processing_time=('ttime', lambda x: round(np.mean(x) + 1e-9, 3)))

In [74]:
data = [[0, 0, 'start', 0.712], [0, 0, 'end', 1.52], [0, 1, 'start', 3.14], [0, 1, 'end', 4.12], [1, 0, 'start', 0.55], [1, 0, 'end', 1.55], [1, 1, 'start', 0.43], [1, 1, 'end', 1.42], [2, 0, 'start', 4.1], [2, 0, 'end', 4.512], [2, 1, 'start', 2.5], [2, 1, 'end', 5]]
activity = pd.DataFrame(data, columns=['machine_id', 'process_id', 'activity_type', 'timestamp']).astype({'machine_id':'Int64', 'process_id':'Int64', 'activity_type':'object', 'timestamp':'Float64'})

In [75]:
get_average_time(activity)

,machine_id,processing_time
0,0,0.894
1,1,0.995
2,2,1.456


## 1731. The Number of Employees Which Report to Each Employee [E]

Table: Employees
```
+-------------+----------+
| Column Name | Type     |
+-------------+----------+
| employee_id | int      |
| name        | varchar  |
| reports_to  | int      |
| age         | int      |
+--------`-----+----------+
employee_id is the column with unique values for this table.
This table contains information about the employees and the id of the manager they report to. Some employees do not report to anyone (reports_to is null). 
```

For this problem, we will consider a manager an employee who has at least 1 other employee reporting to them.

Write a solution to report the ids and the names of all managers, the number of employees who report directly to them, and the average age of the reports rounded to the nearest integer.

Return the result table ordered by employee_id.

The result format is in the following example.

In [103]:
# Beats 5.71%
def count_employees(employees: pd.DataFrame) -> pd.DataFrame:
    managers = employees.copy()
    managers = managers[['employee_id','name','age']]
    managers.columns = ['reports_to','name_m','age_m']
    df = pd.merge(
        employees, managers, how='inner', on='reports_to'
    ).groupby(
        ['reports_to','name_m'], as_index=False
    ).agg(
        reports_count=('employee_id','count'), average_age=('age',lambda x: round(np.mean(x)+1e-9,0),)
    )
    df.columns = ['employee_id','name','reports_count','average_age']
    return df

In [98]:
data = [[9, 'Hercy', None, 43], [6, 'Alice', 9, 41], [4, 'Bob', 9, 36], [2, 'Winston', None, 37]]
employees = pd.DataFrame(data, columns=['employee_id', 'name', 'reports_to', 'age']).astype({'employee_id':'Int64', 'name':'object', 'reports_to':'Int64', 'age':'Int64'})

In [106]:
count_employees(employees)

,employee_id,name,reports_count,average_age


## 1978. Employees Whose Manager Left the Company [E]

Table: Employees
```
+-------------+----------+
| Column Name | Type     |
+-------------+----------+
| employee_id | int      |
| name        | varchar  |
| manager_id  | int      |
| salary      | int      |
+-------------+----------+
In SQL, employee_id is the primary key for this table.
This table contains information about the employees, their salary, and the ID of their manager. Some employees do not have a manager (manager_id is null). 
```

Find the IDs of the employees whose salary is strictly less than $30000 and whose manager left the company. When a manager leaves the company, their information is deleted from the Employees table, but the reports still have their manager_id set to the manager that left.

Return the result table ordered by employee_id.

The result format is in the following example.

In [114]:
# Beats 62.96%
def find_employees(employees: pd.DataFrame) -> pd.DataFrame:
    return employees.loc[ (~employees['manager_id'].isin(employees['employee_id'])) & (~employees['manager_id'].isna()) & (employees['salary']<30000), ['employee_id']].sort_values('employee_id')

In [118]:
data = [[3, 'Mila', 9, 60301], [12, 'Antonella', None, 31000], [13, 'Emery', None, 67084], [1, 'Kalel', 11, 21241], [9, 'Mikaela', None, 50937], [11, 'Joziah', 6, 28485]]
employees = pd.DataFrame(data, columns=['employee_id', 'name', 'manager_id', 'salary']).astype({'employee_id':'Int64', 'name':'object', 'manager_id':'Int64', 'salary':'Int64'})

In [119]:
find_employees(employees)

,employee_id
5,11


In [108]:
employees

,employee_id,name,manager_id,salary
0,3,Mila,9,60301
1,12,Antonella,<NA>,31000
2,13,Emery,<NA>,67084
3,1,Kalel,11,21241
4,9,Mikaela,<NA>,50937
5,11,Joziah,6,28485


In [113]:
employees.loc[ (~employees['manager_id'].isin(employees['employee_id'])) & (~employees['manager_id'].isna()) & (employees['salary']<30000), ['employee_id']].sort_values('employee_id')

,employee_id
5,11
